In [30]:
import numpy as np
import tensorflow as tf
import os
import sklearn
import keras
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf
import os
import sklearn
import keras
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from training_utils import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer

In [31]:
X_train = np.load("train_processed_imgs.npy")
y_train_original = np.load("train_labels.npy")
X_val = np.load("val_processed_imgs.npy")
y_val = np.load("val_labels.npy")

In [54]:
training_1 = X_train[0:1000]
training_2 = X_train[1000:2000]
training_3 = X_train[2000:]

In [55]:
training_1.shape
training_2.shape

(1000, 256, 256, 3)

In [58]:
test_train = np.concatenate((training_1, training_2, training_3), axis=0)
test_train.shape

(2681, 256, 256, 3)

In [29]:
np.savetxt("testing_data.csv", X_train, delimiter=",")

ValueError: Expected 1D or 2D array, got 4D array instead

In [4]:
y_true = y_val

In [5]:
label_as_binary = LabelBinarizer()
y_train = label_as_binary.fit_transform(y_train_original)
y_val = label_as_binary.fit_transform(y_val)

In [ ]:
datasets = {'X_train': X_train, 'y_train': y_train, 'X_val': X_val, 'y_val': y_val}

In [3]:
model_parameters = {
    'layers': [
        {'type': 'Rescaling', 'scaling_factor': 1./255},
        {'type': 'Conv2D', 'filters': 4, 'kernel_size': 11, 'activation': 'relu'},
        {'type': 'MaxPooling2D'},
        {'type': 'Conv2D', 'filters': 8, 'kernel_size': 7, 'activation': 'relu'},
        {'type': 'MaxPooling2D'},
        {'type': 'Flatten'},
        {'type': 'Dense', 'units': 20, 'activation': 'relu'},
        {'type': 'Dropout', 'rate': 0.3},
        {'type': 'Dense', 'units': 4, 'activation': 'softmax'}
    ],
    
    'hyperparams' : 
        {'optimizer': 'Adam',
        'learning_rate': 0.01,
        'decay': None,
        'batch_size': 64,
        'loss_func': 'categorical_crossentropy',
        'metrics': 'acc'},
    
    'training_config' : 
        {'epochs': 3,
         'batch_size': 32,
         'class_weights': None}
}

In [ ]:
CNN = CNNModel.new_instance(model_parameters)
(experimentID, runID) = CNN.mlflow_run(datasets)

In [3]:
num_classes = 4

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(4, 11, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(8, 7, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(20, activation='relu', kernel_regularizer='l2'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(4, activation = 'softmax')
])

model.build(input_shape=(None, 256, 256, 3))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 246, 246, 4)       1456      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 123, 123, 4)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 117, 117, 8)       1576      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 58, 58, 8)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 26912)            

In [71]:
# class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train_original),
#                                      y=y_train_original)